# Data for Dashboard

## 0. Data Load

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/기상청/데이터')

In [3]:
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
from tqdm.notebook import tqdm
tqdm.pandas()

import random
from functools import reduce

import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
%matplotlib inline

rc('font', family='MalgunGothic')
plt.rcParams['axes.unicode_minus'] = False

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
%%time
df_st = pd.read_csv('stationary_causality.csv', encoding='CP949')
df_final = pd.read_csv('final_0622.csv', encoding='CP949')
df_result1 = pd.read_csv('df_pred.csv', encoding='CP949')
df_result2 = pd.read_csv('df_pred_jb.csv', encoding='CP949')
df_result3 = pd.read_csv('df_pred_hr.csv', encoding='CP949')
df_before = pd.read_csv('final_categorical_0622.csv', encoding='CP949')
df_non = pd.read_csv('nonst_high_for_dashboard_0622.csv', encoding='CP949')
df_clust = pd.read_csv('tsclustering_0620.csv', encoding='CP949')

CPU times: user 51.8 s, sys: 5.42 s, total: 57.2 s
Wall time: 1min 9s


In [ ]:
df_all = pd.read_csv('0623-rec.csv', encoding='CP949')
df_mine = pd.read_csv('df_0623.csv', encoding='CP949')

In [ ]:
df_again = pd.read_csv('df_0624.csv', encoding='CP949')

In [ ]:
df_st.head(2)

,sm_cat,avg_ta,hm_max,max_pa,max_ta,min_ta,PM10,PM25,avg_ws,sum_ss_hr,total
0,가스온수기,None,None,None,None,None,causality,None,None,None,1
1,가자미,causality,None,None,None,None,None,None,None,None,1


In [ ]:
df_final.head(2)

,date,sex,big_cat,sm_cat,qty,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa,공휴일여부,주말여부,연_2019,월_2,월_3,월_4,월_5,월_6,월_7,월_8,월_9,월_10,월_11,월_12,분기_2,분기_3,분기_4,요일_1,요일_2,요일_3,요일_4,요일_5,요일_6,봄,여름,가을,겨울,cnt,age_20,age_30,age_40,age_50,age_60
0,2018-01-01,1,식품,가공란,37.0,0.0,10.69387,4.09137,0.0,0.00000,0.04440,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.480964,1,0,0,0,0
1,2018-01-01,1,식품,가공란,16.0,0.0,12.42969,6.35294,0.0,0.51282,0.42417,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.480964,0,1,0,0,0


In [ ]:
df_result1.head(2)

,date,sex,age,sm_cat,pred
0,2019-10-03,남성,20대,샤워코롱,0.089238
1,2019-10-04,남성,20대,샤워코롱,0.093895


In [ ]:
df_result1['sm_cat'].unique()

array(['샤워코롱', '선크림', '소고기 등심/안심', '수산 생물', '스킨케어 코팩', '아몬드유/코코넛밀크',
       '에이드', '옻/칡/쑥즙', '의류건조기', '이온음료', '인스턴트커피', '전복 생물', '절임배추/김치속',
       '젓갈류', '제습기', '차/곡물 음료', '천장형 에어컨', '카페 푸드', '클렌징 크림', '클렌징 폼',
       '태닝용 선크림', '팩도구', '표고버섯', '풋워시/스크럽', '풋크림', '해조류 다시마',
       '헤어스타일링용 펌제', '헤어케어세트', '홍어', '화장 비누', '멀티형 에어컨'], dtype=object)

In [ ]:
df_result2.head(2)

,date,sex,age,sm_cat,pred
0,2019-10-03,남성,20대,녹차,2.127701
1,2019-10-04,남성,20대,녹차,1.642287


In [ ]:
df_result2['sm_cat'].unique()

array(['녹차', '닭 양념육', '더치커피', '돈풍기', '랍스타', '레몬/자몽', '린스', '메이크업 박스',
       '무/배추', '바나나/파인애플/망고', '바디 세트', '반건조고구마', '배/포도/과일즙', '백김치',
       '베이스 메이크업용 BB크림', '베이스 메이크업용 CC크림', '베이스 메이크업용 메이크업베이스',
       '벽걸이형 냉온풍기', '보일러', '뷰티 속눈썹/쌍꺼풀', '뷰티용 기름종이', '뷰티용 뷰러',
       '뷰티용 여드름압출기', '삼치', '색조 메이크업 립글로스', '색조 메이크업 마스카라', '색조 메이크업 볼터치',
       '색조 메이크업 쉐딩/하이라이터', '생수'], dtype=object)

In [ ]:
df_result3.head(2)

,date,sex,age,sm_cat,pred
0,2019-10-06,남성,20대,가스온수기,0.0
1,2019-10-07,남성,20대,가스온수기,0.0


In [ ]:
df_result3['sm_cat'].unique()

array(['가스온수기', '가자미', '감말랭이', '갓김치', '건강즙/녹용', '건어물 멸치', '건어물 황태', '건자두',
       '견과류', '견과류 땅콩', '견과류 마카다미아', '견과류 잣/은행', '견과류 호두', '공기정화 용품',
       '공기청정기', '과채 음료/주스', '기능성 링클케어 화장품', '기능성 영양보습 화장품',
       '기초 화장용 오일/앰플', '기타 농산물', '기타 한방/환제품', '나물', '남성 스킨', '남성 크림',
       '남성 클렌징', '네일리무버', '네일세트', '네일아트', '네일컬러'], dtype=object)

In [ ]:
df_result1['sm_cat'].nunique()+df_result2['sm_cat'].nunique()+df_result3['sm_cat'].nunique()

89

In [ ]:
df_before.head(2)

,date,sex,age,big_cat,sm_cat,qty,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa,연,월,분기,요일,공휴일여부,주말여부,계절,cnt
0,2018-01-01,F,20,식품,가공란,37.0,0.0,10.69387,4.09137,0.0,0.00000,0.04440,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,2018,1,1,0,1,0,3,0.480964
1,2018-01-01,F,30,식품,가공란,16.0,0.0,12.42969,6.35294,0.0,0.51282,0.42417,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,2018,1,1,0,1,0,3,0.480964


In [ ]:
df_non.head(2)

,date,sex,age,big_cat,sm_cat,qty,cnt,날씨,영향력,pred,상품분류
0,2018-01-01,F,30,식품,건대추,8.0,0.098046,미세먼지,None,NaN,계절상품
1,2018-01-01,F,40,식품,초코우유,1.0,7.310201,최고기온,None,NaN,계절상품


In [ ]:
df_non.shape

(3856716, 11)

In [ ]:
df_all.head(2)

In [ ]:
df_mine.head(2)

,date,sex,age,big_cat,sm_cat,qty,cnt,날씨,영향력,pred,상품분류
0,2018-01-01,여성,30대,식품,건대추,8.0,0.098046,미세먼지,None,NaN,날씨와 인과관계 있는 계절상품
1,2018-01-01,여성,40대,식품,초코우유,1.0,7.310201,최고기온,None,NaN,날씨와 인과관계 있는 계절상품


In [ ]:
df_again.head(2)

,age,sex,month,sm_cat,rain,dust,rank,rec,avg_ta,hm_max,max_pa,max_ta,min_ta,PM10,PM25,avg_ws,sum_ss_hr,total,var_lag
0,20대,여성,1,가스온수기,맑은 날,미세먼지 좋은 날,1,바디 클렌져,None,None,None,None,None,None,None,None,None,0,29
1,20대,여성,1,가스온수기,맑은 날,미세먼지 좋은 날,2,바디 보습제,None,None,None,None,None,None,None,None,None,0,29


In [ ]:
df_clust.head(2)

,sm_cat,01-01 00:00:00,01-02 00:00:00,01-03 00:00:00,01-04 00:00:00,01-05 00:00:00,01-06 00:00:00,01-07 00:00:00,01-08 00:00:00,01-09 00:00:00,01-10 00:00:00,01-11 00:00:00,01-12 00:00:00,01-13 00:00:00,01-14 00:00:00,01-15 00:00:00,01-16 00:00:00,01-17 00:00:00,01-18 00:00:00,01-19 00:00:00,01-20 00:00:00,01-21 00:00:00,01-22 00:00:00,01-23 00:00:00,01-24 00:00:00,01-25 00:00:00,01-26 00:00:00,01-27 00:00:00,01-28 00:00:00,01-29 00:00:00,01-30 00:00:00,01-31 00:00:00,02-01 00:00:00,02-02 00:00:00,02-03 00:00:00,02-04 00:00:00,02-05 00:00:00,02-06 00:00:00,02-07 00:00:00,02-08 00:00:00,...,11-23 00:00:00,11-24 00:00:00,11-25 00:00:00,11-26 00:00:00,11-27 00:00:00,11-28 00:00:00,11-29 00:00:00,11-30 00:00:00,12-01 00:00:00,12-02 00:00:00,12-03 00:00:00,12-04 00:00:00,12-05 00:00:00,12-06 00:00:00,12-07 00:00:00,12-08 00:00:00,12-09 00:00:00,12-10 00:00:00,12-11 00:00:00,12-12 00:00:00,12-13 00:00:00,12-14 00:00:00,12-15 00:00:00,12-16 00:00:00,12-17 00:00:00,12-18 00:00:00,12-19 00:00:00,12-20 00:00:00,12-21 00:00:00,12-22 00:00:00,12-23 00:00:00,12-24 00:00:00,12-25 00:00:00,12-26 00:00:00,12-27 00:00:00,12-28 00:00:00,12-29 00:00:00,12-30 00:00:00,12-31 00:00:00,cluster
0,가열식 가습기,1.402818,0.69424,1.204881,1.127676,0.291099,1.167524,0.540680,1.830916,0.964416,0.495974,0.313917,1.020381,0.330088,0.690992,0.285643,0.472651,0.056057,0.353435,-0.113052,0.035968,0.306146,0.377411,0.465706,0.710751,0.853612,-0.271786,0.531928,0.532093,1.234474,0.472651,0.323962,0.115004,0.293827,-0.004047,-0.337884,-0.265957,-0.003716,0.443012,0.472651,...,0.805661,0.894101,1.425715,0.826619,1.387706,1.629423,1.330475,0.890236,0.711082,1.843218,1.885736,1.673075,2.643134,3.843077,1.436090,1.666086,2.180527,2.840078,1.778174,1.868020,1.408211,1.194790,0.832238,1.868516,1.306251,0.651474,1.515457,0.892272,0.901827,1.057486,1.307066,1.128006,0.651061,1.365776,1.178210,1.068068,1.256275,0.994488,0.404693,1
1,갈비/찜/바비큐용 돈육,-0.388040,-0.65423,-0.757428,-1.341054,-0.269082,-0.968768,-0.673161,0.193074,-0.542894,-0.546280,-0.668534,0.488537,0.301444,-0.245335,-0.007709,-0.379934,-0.060688,-0.715343,0.166584,-0.651902,-1.593862,-0.229607,-0.292855,1.143335,-0.578001,-0.855132,0.122547,0.744486,-1.066376,0.094519,-1.201103,-1.106043,-0.335238,0.044934,-0.258386,-0.090249,-0.271764,-0.564262,1.215078,...,-0.356151,0.232472,1.094044,-0.395828,0.171815,-0.039285,0.530987,0.351338,0.546722,-0.617533,0.474540,1.340998,0.747001,0.182478,0.326705,0.325017,-0.157295,-0.356151,0.304625,0.066193,-0.015639,1.048175,0.736719,1.830167,1.074013,0.720847,0.332863,-0.142436,-0.213655,1.516286,-0.128111,0.372345,1.245824,0.388289,-0.324508,1.254188,0.088368,0.781504,1.420467,2


## 1. Prediction

### 1-1. 비정상

In [ ]:
df_cluster = df_clust.loc[:,['sm_cat','cluster']]

In [ ]:
def cluster(number):
    if number == 0:
        return '날씨와 인과관계 있는 여름상품'
    elif number == 1:
        return '날씨와 인과관계 있는 겨울상품'
    else:
        return '날씨와 인과관계 있는 변동성이 큰 스테디셀러'

In [ ]:
df_cluster['cluster'] = df_cluster['cluster'].apply(cluster)

In [ ]:
df_cluster['cluster'].unique()

array(['날씨와 인과관계 있는 겨울상품', '날씨와 인과관계 있는 변동성이 큰 스테디셀러', '날씨와 인과관계 있는 여름상품'],
      dtype=object)

In [ ]:
print(len(df_cluster[df_cluster['cluster']=='날씨와 인과관계 있는 겨울상품']))
print(len(df_cluster[df_cluster['cluster']=='날씨와 인과관계 있는 여름상품']))
print(len(df_cluster[df_cluster['cluster']=='날씨와 인과관계 있는 변동성이 큰 스테디셀러']))

52
51
31


In [ ]:
winter = df_cluster[df_cluster['cluster']=='날씨와 인과관계 있는 겨울상품']['sm_cat'].tolist()
summer = df_cluster[df_cluster['cluster']=='날씨와 인과관계 있는 여름상품']['sm_cat'].tolist()
steady = df_cluster[df_cluster['cluster']=='날씨와 인과관계 있는 변동성이 큰 스테디셀러']['sm_cat'].tolist()

In [ ]:
def rename(classification):
    if classification in summer:
        return '날씨와 인과관계 있는 여름상품'
    elif classification in winter:
        return '날씨와 인과관계 있는 겨울상품'
    else:
        return '날씨와 인과관계 있는 변동성이 큰 스테디셀러'

In [ ]:
df_non.drop(['상품분류'], axis=1, inplace=True)

In [ ]:
df_non['상품분류'] = df_non['sm_cat'].apply(rename)

In [ ]:
df_non['date'] = pd.to_datetime(df_non['date'], format='%Y.%m.%d')
df_non['date'] = df_non['date'].dt.date

In [ ]:
df_non['sex'] = df_non['sex'].apply(lambda x: '여성' if x=='F' else '남성')
df_non['age'] = df_non['age'].apply(lambda x: str(x)+'대')

In [ ]:
df_non.head(2)

,date,sex,age,big_cat,sm_cat,qty,cnt,날씨,영향력,pred,상품분류
0,2018-01-01,여성,30대,식품,건대추,8.0,0.098046,미세먼지,None,NaN,날씨와 인과관계 있는 여름상품
1,2018-01-01,여성,40대,식품,초코우유,1.0,7.310201,최고기온,None,NaN,날씨와 인과관계 있는 여름상품


## 1-2. 정상

In [ ]:
# LSTM 결과
df_result = pd.concat([df_result1, df_result2, df_result3], axis=0).reset_index(drop=True)

In [ ]:
df_result['sex'] = df_result['sex'].apply(lambda x: x[:2] if x=='여성 ' else x)

In [ ]:
df_result['sex'].unique()

array(['남성', '여성'], dtype=object)

In [ ]:
# 사용할 변수
df_use = df_before[['date', 'sex', 'age', 'big_cat', 'sm_cat', 'qty', 'cnt']]

In [ ]:
df_before.columns

Index(['date', 'sex', 'age', 'big_cat', 'sm_cat', 'qty', 'ratio', '오늘날씨',
       '미세먼지', '오늘 비', '기상정보', '태풍정보', 'avg_ta', 'max_ta', 'min_ta', 'rn_day',
       'rn_hr1', 'avg_ws', 'PM10', 'PM25', 'hm_max', 'sum_ss_hr', 'max_pa',
       '연', '월', '분기', '요일', '공휴일여부', '주말여부', '계절', 'cnt'],
      dtype='object')

In [ ]:
df_use.head(2)

,date,sex,age,big_cat,sm_cat,qty,cnt
0,2018-01-01,F,20,식품,가공란,37.0,0.480964
1,2018-01-01,F,30,식품,가공란,16.0,0.480964


In [ ]:
df_st.columns

Index(['sm_cat', 'avg_ta', 'hm_max', 'max_pa', 'max_ta', 'min_ta', 'PM10',
       'PM25', 'avg_ws', 'sum_ss_hr', 'total'],
      dtype='object')

In [ ]:
# 한글로 변경
df_st = df_st.rename(columns={"avg_ta" : "평균기온", "hm_max": "최고습도", "max_pa": "최고기압", "max_ta": "최고기온", "min_ta": "최저기온",
                              "PM10": "미세먼지", "PM25": "초미세먼지", "avg_ws": "평균풍속", "sum_ss_hr": "일조시간합"})

In [ ]:
# 날씨에 따른 영향력
df_causal = pd.melt(df_st, id_vars=['sm_cat'])
df_causality = df_causal[df_causal['variable']!='total']
df_causality = df_causality.rename(columns={"variable": "날씨", "value": "영향력"})

In [ ]:
df_causality.head(2)

,sm_cat,날씨,영향력
0,가스온수기,평균기온,None
1,가자미,평균기온,causality


In [ ]:
df_tb = pd.merge(df_use, df_causality, how='left', on='sm_cat')

In [ ]:
df_tb.head(2)

,date,sex,age,big_cat,sm_cat,qty,cnt,날씨,영향력
0,2018-01-01,F,20,식품,가공란,37.0,0.480964,NaN,NaN
1,2018-01-01,F,30,식품,가공란,16.0,0.480964,NaN,NaN


In [ ]:
df_tb['sex'] = df_tb['sex'].apply(lambda x: '여성' if x=='F' else '남성')
df_tb['age'] = df_tb['age'].apply(lambda x: str(x)+'대')

In [ ]:
df_tb.head(2)

,date,sex,age,big_cat,sm_cat,qty,cnt,날씨,영향력
0,2018-01-01,여성,20대,식품,가공란,37.0,0.480964,NaN,NaN
1,2018-01-01,여성,30대,식품,가공란,16.0,0.480964,NaN,NaN


In [ ]:
df_result.head(2)

,date,sex,age,sm_cat,pred
0,2019-10-03,남성,20대,샤워코롱,0.089238
1,2019-10-04,남성,20대,샤워코롱,0.093895


In [ ]:
df_result['sex'].unique()

array(['남성', '여성'], dtype=object)

In [ ]:
# 원래 데이터에 예측 데이터 병합
df_ex = pd.merge(df_tb, df_result, how='left', on=['date', 'sex', 'age', 'sm_cat'])

In [ ]:
# 예측한 소분류만
df_isin = df_ex[df_ex.sm_cat.isin(df_result['sm_cat'])].reset_index(drop=True)

In [ ]:
df_isin.shape

(5349375, 10)

In [ ]:
df_isin.isnull().sum()

date             0
sex              0
age              0
big_cat          0
sm_cat           0
qty              0
cnt        1077075
날씨               0
영향력              0
pred       4729167
dtype: int64

In [ ]:
# cnt 결측값 제거
df_isin['cnt'] = df_isin['cnt'].fillna(0)

In [ ]:
df_isin.head(2)

,date,sex,age,big_cat,sm_cat,qty,cnt,날씨,영향력,pred
0,2018-01-01,여성,20대,냉난방가전,가스온수기,0.0,0.0,평균기온,None,NaN
1,2018-01-01,여성,20대,냉난방가전,가스온수기,0.0,0.0,최고습도,None,NaN


In [ ]:
df_isin['상품분류'] = '날씨와 인과관계 있는 변동성이 작은 스테디셀러'

In [ ]:
df_isin.head(2)

,date,sex,age,big_cat,sm_cat,qty,cnt,날씨,영향력,pred,상품분류
0,2018-01-01,여성,20대,냉난방가전,가스온수기,0.0,0.0,평균기온,None,NaN,날씨와 인과관계 있는 변동성이 작은 스테디셀러
1,2018-01-01,여성,20대,냉난방가전,가스온수기,0.0,0.0,최고습도,None,NaN,날씨와 인과관계 있는 변동성이 작은 스테디셀러


In [ ]:
df_isin.to_csv('df_isin_0623.csv', encoding='CP949', index=False)

## 1-3. 병합

In [ ]:
print(df_non.shape)
print(df_isin.shape)

(3856716, 11)
(5349375, 11)


In [ ]:
df_concat = pd.concat([df_non, df_isin], axis=0).reset_index(drop=True)

In [ ]:
df_concat.head(2)

,date,sex,age,big_cat,sm_cat,qty,cnt,날씨,영향력,pred,상품분류
0,2018-01-01,여성,30대,식품,건대추,8.0,0.098046,미세먼지,None,NaN,날씨와 인과관계 있는 여름상품
1,2018-01-01,여성,40대,식품,초코우유,1.0,7.310201,최고기온,None,NaN,날씨와 인과관계 있는 여름상품


In [ ]:
df_concat.to_csv('df_result_0624.csv', encoding='CP949', index=False)

## 2. Recommendation system

In [ ]:
df_mine.head(2)

,date,sex,age,big_cat,sm_cat,qty,cnt,날씨,영향력,pred,상품분류
0,2018-01-01,여성,30대,식품,건대추,8.0,0.098046,미세먼지,None,NaN,날씨와 인과관계 있는 계절상품
1,2018-01-01,여성,40대,식품,초코우유,1.0,7.310201,최고기온,None,NaN,날씨와 인과관계 있는 계절상품


In [ ]:
df_all.head(2)

,Unnamed: 0,age,sex,month,sm_cat,rain,dust,rank,rec,avg_ta,hm_max,max_pa,max_ta,min_ta,PM10,PM25,avg_ws,sum_ss_hr,total,var_lag
0,0,20,F,1,가스온수기,맑은 날,미세먼지 좋은 날,1,바디 클렌져,None,None,None,None,None,None,None,None,None,0,29
1,1,20,F,1,가스온수기,맑은 날,미세먼지 좋은 날,2,바디 보습제,None,None,None,None,None,None,None,None,None,0,29


In [ ]:
df_mine['month'] = df_mine['date'].apply(lambda x: int(x[5:7]))

In [ ]:
df_mine['sex'] = df_mine['sex'].apply(lambda x: 'F' if x=='여성' else 'M')

In [ ]:
df_mine['age'] = df_mine['age'].apply(lambda x: x[:2])

In [ ]:
df_mine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7438383 entries, 0 to 7438382
Data columns (total 12 columns):
 #   Column   Dtype  
---  ------   -----  
 0   date     object 
 1   sex      object 
 2   age      object 
 3   big_cat  object 
 4   sm_cat   object 
 5   qty      float64
 6   cnt      float64
 7   날씨       object 
 8   영향력      object 
 9   pred     float64
 10  상품분류     object 
 11  month    int64  
dtypes: float64(3), int64(1), object(8)
memory usage: 681.0+ MB


In [ ]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1267200 entries, 0 to 1267199
Data columns (total 20 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Unnamed: 0  1267200 non-null  int64 
 1   age         1267200 non-null  int64 
 2   sex         1267200 non-null  object
 3   month       1267200 non-null  int64 
 4   sm_cat      1267200 non-null  object
 5   rain        1267200 non-null  object
 6   dust        1267200 non-null  object
 7   rank        1267200 non-null  int64 
 8   rec         1262130 non-null  object
 9   avg_ta      1267200 non-null  object
 10  hm_max      1267200 non-null  object
 11  max_pa      1267200 non-null  object
 12  max_ta      1267200 non-null  object
 13  min_ta      1267200 non-null  object
 14  PM10        1267200 non-null  object
 15  PM25        1267200 non-null  object
 16  avg_ws      1267200 non-null  object
 17  sum_ss_hr   1267200 non-null  object
 18  total       1267200 non-null  int64 
 19  

In [ ]:
df_all['age'] = df_all['age'].apply(lambda x: str(x))

In [ ]:
df_all.columns

Index(['Unnamed: 0', 'age', 'sex', 'month', 'sm_cat', 'rain', 'dust', 'rank',
       'rec', 'avg_ta', 'hm_max', 'max_pa', 'max_ta', 'min_ta', 'PM10', 'PM25',
       'avg_ws', 'sum_ss_hr', 'total', 'var_lag'],
      dtype='object')

In [ ]:
df_all.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df_all['age'] = df_all['age'].apply(lambda x: str(x)+'대')

In [ ]:
df_all['sex'] = df_all['sex'].apply(lambda x: '여성' if x=='F' else '남성')

In [ ]:
df_all.head(2)

,age,sex,month,sm_cat,rain,dust,rank,rec,avg_ta,hm_max,max_pa,max_ta,min_ta,PM10,PM25,avg_ws,sum_ss_hr,total,var_lag
0,20대,여성,1,가스온수기,맑은 날,미세먼지 좋은 날,1,바디 클렌져,None,None,None,None,None,None,None,None,None,0,29
1,20대,여성,1,가스온수기,맑은 날,미세먼지 좋은 날,2,바디 보습제,None,None,None,None,None,None,None,None,None,0,29


In [ ]:
df_all.to_csv('df_0624.csv', encoding='CP949', index=False)